Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 7.22 ms, sys: 1.21 ms, total: 8.43 ms
Wall time: 13.3 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://10.1.0.144:8787/status,
Dashboard: http://10.1.0.144:8787/status,Workers: 1
Total threads: 6,Total memory: 6.78 GiB
Status: running,Using processes: False
Comm: inproc://10.1.0.144/7953/1,Workers: 1
Dashboard: http://10.1.0.144:8787/status,Total threads: 6
Started: Just now,Total memory: 6.78 GiB
Comm: inproc://10.1.0.144/7953/4,Total threads: 6
Dashboard: http://10.1.0.144:43819/status,Memory: 6.78 GiB
Nanny: None,


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 856 ms, sys: 68.1 ms, total: 924 ms
Wall time: 2.48 s


[1.685472210033744,
 1.706428662794123,
 1.0590059685986661,
 1.4525199838109164,
 1.0741507652965365,
 1.357760607695251,
 1.552978999551406,
 1.9078950265868446,
 1.866721173899434,
 1.8529208214757347,
 1.9901607912133061,
 1.2452928787573745,
 1.064999683278475,
 1.3735944699481921,
 1.8444548049927971,
 1.734876176716579,
 1.463108529781536,
 1.032535988176116,
 1.7844084021379256,
 1.4185110374478012,
 1.8550291184426588,
 1.180461332986571,
 1.0206081629254353,
 1.0724946860668059,
 1.1353085458153664,
 1.3227724339701308,
 1.4121245023486293,
 1.7779435103197747,
 1.4078827024328877,
 1.3247553631411428,
 1.3239240839086923,
 1.6980561029322752,
 1.6772282856899339,
 1.3237521032346322,
 1.9585829738150953,
 1.6246655306830777,
 1.5026554753828794,
 1.9576890848332944,
 1.4884235322633736,
 1.7875902147274922,
 1.9110740622115427,
 1.1532172324882022,
 1.5475989215719128,
 1.2583489186556962,
 1.762170331557655,
 1.5683650653166796,
 1.2298030897216579,
 1.4254589260139983,
 1.

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.938308811558847,
 1.8774377249882415,
 1.238916239861374,
 1.3261471005651837,
 1.3690400182390818,
 1.454960287483527,
 1.8765376940743739,
 1.549276943414084,
 1.032124870479214,
 1.3658229531090214,
 1.3433057746478743,
 1.386264014165982,
 1.1791370741068345,
 1.961635116343159,
 1.598679164506951,
 1.176458750000697,
 1.6449494314820123,
 1.776914916960318,
 1.6936796372538865,
 1.075892029561527,
 1.581812279384253,
 1.9532794246932328,
 1.938506746738299,
 1.2100001272057708,
 1.2022271066461012,
 1.3530602100569193,
 1.9693881510918692,
 1.943960356288164,
 1.693301376687506,
 1.104578925367179,
 1.0070332249693812,
 1.041476486115523,
 1.1820298703309717,
 1.8137861555163743,
 1.7776947248077417,
 1.9099424258834463,
 1.5376925227400418,
 1.5007645503616436,
 1.0880728061733618,
 1.4074827066565359,
 1.8725688346168543,
 1.039785376063414,
 1.2269308103512913,
 1.325979790009809,
 1.0123928603155932,
 1.155942158490515,
 1.0469645678456954,
 1.9281616243650523,
 1.324821227

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 315 ms, sys: 42.2 ms, total: 357 ms
Wall time: 2.75 s
